In [40]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

FIGSIZE=(20,10)

In [41]:
data = pd.read_csv('../data/train.csv')

In [42]:
test = pd.read_csv('../data/test.csv')

In [43]:
data.shape, test.shape

((4000, 287), (1686, 286))

In [44]:
colna_tr = data.isna().sum() > 0
colna_tr = colna_tr[colna_tr].index.tolist()

colna_tt = test.isna().sum() > 0
colna_tt = colna_tt[colna_tt].index.tolist()

In [45]:
len(colna_tr), len(colna_tt)

(0, 0)

## Filtering

### Looking on data, trying to reduce the number of features by filtering out non-variative ones

In [46]:
catbytype_cols = data.select_dtypes(include=['object']).columns.tolist()

In [47]:
cols_use = set(data.columns) - set(['Id', 'Energy_consumption']) - set(catbytype_cols)

In [48]:
cols_freq = list(zip(cols_use, map(lambda x: data[x].nunique(), cols_use)))
cols_freq = [(x[0], x[1], x[1]/data.shape[0]) for x in cols_freq]
cols_freq = sorted(cols_freq, key=lambda x: x[1], reverse=True)

In [49]:
len(cols_freq)

281

In [50]:
cols_freq

[('feature_253', 3740, 0.935),
 ('feature_262', 3266, 0.8165),
 ('feature_259', 2985, 0.74625),
 ('feature_260', 2952, 0.738),
 ('feature_264', 2715, 0.67875),
 ('feature_255', 2379, 0.59475),
 ('feature_268', 2305, 0.57625),
 ('feature_254', 2242, 0.5605),
 ('feature_261', 2167, 0.54175),
 ('feature_269', 2005, 0.50125),
 ('feature_270', 1912, 0.478),
 ('feature_250', 1865, 0.46625),
 ('feature_248', 1853, 0.46325),
 ('feature_249', 1815, 0.45375),
 ('feature_265', 1456, 0.364),
 ('feature_271', 1277, 0.31925),
 ('feature_251', 1219, 0.30475),
 ('feature_256', 718, 0.1795),
 ('feature_252', 679, 0.16975),
 ('feature_272', 600, 0.15),
 ('feature_274', 402, 0.1005),
 ('feature_273', 267, 0.06675),
 ('feature_275', 264, 0.066),
 ('feature_278', 217, 0.05425),
 ('feature_277', 215, 0.05375),
 ('feature_267', 215, 0.05375),
 ('feature_279', 210, 0.0525),
 ('feature_276', 155, 0.03875),
 ('feature_266', 136, 0.034),
 ('feature_200', 68, 0.017),
 ('feature_68', 47, 0.01175),
 ('feature_280',

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 287 entries, Id to Energy_consumption
dtypes: float64(22), int64(261), object(4)
memory usage: 8.8+ MB


In [52]:
cols_int = data.select_dtypes(include=['int']).columns.tolist()
cols_float = set(cols_use) - set(cols_int)

In [53]:
cols_int_freq = list(filter(lambda x: x[0] in cols_int, cols_freq))
cols_float_freq = list(filter(lambda x: x[0] in cols_float, cols_freq))

In [54]:
len(cols_float_freq), len(cols_int_freq)

(21, 260)

In [55]:
cols_float_freq

[('feature_253', 3740, 0.935),
 ('feature_262', 3266, 0.8165),
 ('feature_264', 2715, 0.67875),
 ('feature_268', 2305, 0.57625),
 ('feature_269', 2005, 0.50125),
 ('feature_270', 1912, 0.478),
 ('feature_265', 1456, 0.364),
 ('feature_271', 1277, 0.31925),
 ('feature_272', 600, 0.15),
 ('feature_274', 402, 0.1005),
 ('feature_273', 267, 0.06675),
 ('feature_275', 264, 0.066),
 ('feature_278', 217, 0.05425),
 ('feature_277', 215, 0.05375),
 ('feature_267', 215, 0.05375),
 ('feature_279', 210, 0.0525),
 ('feature_276', 155, 0.03875),
 ('feature_266', 136, 0.034),
 ('feature_280', 44, 0.011),
 ('feature_263', 38, 0.0095),
 ('feature_281', 28, 0.007)]

In [56]:
# drop features with only unique value
cols_drop = [x[0] for x in cols_int_freq if x[1] == 1]

In [57]:
cols_drop

['feature_226', 'feature_231']

### Filter out features with low variability

In [58]:
get_variability = lambda x: x.value_counts(normalize=True).iloc[0]
col_var = list(zip(cols_use, map(lambda x: get_variability(data[x]), cols_use)))
col_var = sorted(col_var, key=lambda x:x[1], reverse=True)

In [59]:
var_ratio = 0.95
cols_drop = set([x[0] for x in col_var if x[1] >= 0.90]).union(set(cols_drop))

In [60]:
len(cols_drop)

42

In [61]:
data.drop(cols_drop, axis=1, inplace=True)
test.drop(cols_drop, axis=1, inplace=True)

In [62]:
# recompute staff for categorical/numerical distinction by frequency of unique values
cols_use = set(data.columns) - set(['Id', 'Energy_consumption']) - set(catbytype_cols)

cols_freq = list(zip(cols_use, map(lambda x: data[x].nunique(), cols_use)))
cols_freq = sorted(cols_freq, key=lambda x: x[1], reverse=True)
cols_freq = [(x[0], x[1], x[1]/data.shape[0]) for x in cols_freq]

cols_int = data.select_dtypes(include=['int']).columns.tolist()
cols_float = set(cols_use) - set(cols_int)

cols_int_freq = list(filter(lambda x: x[0] in cols_int, cols_freq))
cols_float_freq = list(filter(lambda x: x[0] in cols_float, cols_freq))

In [63]:
cols_int_freq

[('feature_259', 2985, 0.74625),
 ('feature_260', 2952, 0.738),
 ('feature_255', 2379, 0.59475),
 ('feature_254', 2242, 0.5605),
 ('feature_261', 2167, 0.54175),
 ('feature_250', 1865, 0.46625),
 ('feature_248', 1853, 0.46325),
 ('feature_249', 1815, 0.45375),
 ('feature_251', 1219, 0.30475),
 ('feature_256', 718, 0.1795),
 ('feature_252', 679, 0.16975),
 ('feature_200', 68, 0.017),
 ('feature_68', 47, 0.01175),
 ('feature_168', 43, 0.01075),
 ('feature_59', 41, 0.01025),
 ('feature_148', 38, 0.0095),
 ('feature_128', 37, 0.00925),
 ('feature_129', 36, 0.009),
 ('feature_149', 35, 0.00875),
 ('feature_150', 35, 0.00875),
 ('feature_127', 33, 0.00825),
 ('feature_63', 28, 0.007),
 ('feature_92', 25, 0.00625),
 ('feature_86', 24, 0.006),
 ('feature_60', 21, 0.00525),
 ('feature_66', 20, 0.005),
 ('feature_21', 19, 0.00475),
 ('feature_81', 19, 0.00475),
 ('feature_20', 14, 0.0035),
 ('feature_137', 14, 0.0035),
 ('feature_155', 13, 0.00325),
 ('feature_160', 13, 0.00325),
 ('feature_121'

In [64]:
cols_float_freq

[('feature_253', 3740, 0.935),
 ('feature_262', 3266, 0.8165),
 ('feature_264', 2715, 0.67875),
 ('feature_268', 2305, 0.57625),
 ('feature_269', 2005, 0.50125),
 ('feature_270', 1912, 0.478),
 ('feature_265', 1456, 0.364),
 ('feature_271', 1277, 0.31925),
 ('feature_272', 600, 0.15),
 ('feature_274', 402, 0.1005),
 ('feature_267', 215, 0.05375),
 ('feature_266', 136, 0.034),
 ('feature_280', 44, 0.011),
 ('feature_263', 38, 0.0095),
 ('feature_281', 28, 0.007)]

In [65]:
# take as categorical features - those with less than 20 unique values
# we would assume that float columns are numerical by definition
cols_cat = [x[0] for x in cols_int_freq if x[1] <= 20] + catbytype_cols

In [66]:
cols_num = set(cols_use) - set(cols_cat)
cols_num = list(cols_num.union(cols_float))

In [67]:
print("Number numerical: {}\nNumber categorical: {}\nNumber filtered: {}".format(
    len(cols_num), len(cols_cat), len(cols_drop)
))

Number numerical: 40
Number categorical: 203
Number filtered: 42


In [68]:
cols_use = cols_num + cols_cat

In [69]:
len(cols_use)

243

In [70]:
cols_rename_dict = dict(zip(cols_use, ["_".join([col, 'cat' if col in cols_cat else 'num']) 
                                  for col in cols_use]))

In [71]:
data.rename(columns=cols_rename_dict, inplace=True)
test.rename(columns=cols_rename_dict, inplace=True)

In [72]:
data.to_csv('../data/data_improved/train_drop_variable_renamed.csv', index=False)
test.to_csv('../data/data_improved/test_drop_variable_renamed.csv', index=False)